In [ ]:
# Install all necessary libraries
!pip install -q transformers datasets pdfplumber nltk spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2

In [ ]:
import pdfplumber
from google.colab import files

# Upload a PDF
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# Extract text from the PDF
text = ""
with pdfplumber.open(file_name) as pdf:
    for page in pdf.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"

print("📄 Preview of Document Text:\n")
print(text[:1000])  # Preview first 1000 characters


Saving SampleContract-Shuttle.pdf to SampleContract-Shuttle.pdf
📄 Preview of Document Text:

Sample Contract
Contract No.___________
PROFESSIONAL SERVICES AGREEMENT
THIS AGREEMENT made and entered into this _______day of , 20 by and between the SANTA
CRUZ COUNTY REGIONAL TRANSPORTATION COMMISSION, hereinafter called COMMISSION, and
________ ____, hereinafter called CONSULTANT for __________________ (services/project name).
1. DUTIES.
A. CONSULTANT agrees to exercise special skill to accomplish the following results in a manner
reasonably satisfactory to COMMISSION: ______________________________, as specified in Exhibit
A: Scope of Services, which by this reference is incorporated herein.
B. CONSULTANT shall provide the personnel listed below to perform the above-specified services, which
persons are hereby designated as key personnel under this Agreement.
Name Firm Function
Principal in Charge
Project Manager
C. No person named in paragraph B of this Section, or his or her successor, 

In [ ]:
from transformers import pipeline

# Load BART model for summarization
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Function to split long text into chunks
def split_text(text, max_len=1024):
    sentences = text.split('. ')
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) < max_len:
            current_chunk += sentence + ". "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + ". "
    chunks.append(current_chunk.strip())
    return chunks

chunks = split_text(text)
summaries = []

for chunk in chunks[:3]:  # limit to first 3 chunks for demo
    summary = summarizer(chunk, max_length=150, min_length=50, do_sample=False)
    summaries.append(summary[0]['summary_text'])

# Combine all summaries
final_summary = "\n".join(summaries)
print("\n🧠 Final Summary:\n")
print(final_summary)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu



🧠 Final Summary:

The Agreement is between the Santa Cruz County Regional Transportation Commission and the Consultant for the project. The Consultant shall provide the personnel listed below to perform the above-specified services, which are hereby designated as key personnel under this Agreement. The Contract is available for download at the bottom of the page.
No person named in paragraph B of this Section, or his or her successor, shall be removed or replaced by                CONSULTANT. No function hereunder be changed, without the prior written consent of COMMISSION. The CONSULTANT’s Project Manager shall meet with the COMMISSION’S Contract Manager, asneeded, to discuss progress on the contract.
B. Total payment is not to exceed $_____for time and materials. C. Transportation and subsistence costs shall not exceed the rates authorized to employees under current U.S. General Service Administration rules. D. Reimbursable expenses will be billed by CONSULTANT and processed for pay

In [ ]:
import re

# Define important clause keywords
clause_keywords = [
    'confidentiality', 'termination', 'governing law',
    'liability', 'indemnification', 'dispute resolution',
    'intellectual property', 'force majeure'
]

print("\n📌 Detected Clauses:\n")
clause_data = ""

for keyword in clause_keywords:
    pattern = re.compile(rf"([^.]*{keyword}[^.]*\.)", re.IGNORECASE)
    matches = pattern.findall(text)
    if matches:
        print(f"\n🔸 {keyword.upper()} CLAUSE(S):")
        clause_data += f"\n🔸 {keyword.upper()} CLAUSE(S):\n"
        for match in matches:
            print("–", match.strip())
            clause_data += "– " + match.strip() + "\n"



📌 Detected Clauses:


🔸 TERMINATION CLAUSE(S):
– EARLY TERMINATION.
– Within thirty days of the COMMISSION's
receipt of CONSULTANT's final billing, COMMISSION shall pay CONSULTANT its allowable costs
incurred to date of termination and those allowable costs determined by COMMISSION to be
reasonably necessary to effect such termination.
– Such action shall include, but not be limited to, the following: recruitment; advertising, layoff or termination;
rates of pay or other forms of compensation; and selection for training (including apprenticeship),
employment, upgrading, demotion, or transfer.

🔸 LIABILITY CLAUSE(S):
– Any and all claims, demands, losses, damages, defense costs, or liability of any kind or nature which
COMMISSION may sustain or incur or which may be imposed upon it for injury to or death of persons,
or damage to property which arise out of, pertain to, or relate to CONSULTANT’S negligence,
recklessness, or willful misconduct under the terms of this Agreement.
– 2) Auto

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

doc = nlp(text)

print("\n🔍 Named Entities (Parties, Dates, Money, Orgs):")
entities = ""
for ent in doc.ents:
    if ent.label_ in ["ORG", "PERSON", "DATE", "MONEY"]:
        print(f"{ent.text} → {ent.label_}")
        entities += f"{ent.text} → {ent.label_}\n"



🔍 Named Entities (Parties, Dates, Money, Orgs):
COMMISSION → ORG
Exhibit → PERSON
Scope of Services → ORG
COMMISSION → ORG
COMMISSION → ORG
C. Transportation → ORG
U.S. General Service Administration → ORG
D. Reimbursable → ORG
E. Progress → ORG
monthly → DATE
monthly → DATE
COMMISSION → ORG
Contract → ORG
later than 45calendar days → DATE
COMMISSION → ORG
SCCRTC → ORG
1523 → DATE
Pacific Ave → ORG
CA → ORG
95060 → DATE
COMMISSION → ORG
CONSULTANT → ORG
60-calendar days → DATE
the California Labor Code → ORG
COMMISSION → ORG
COMMISSION → ORG
thirty-day → DATE
30-day → DATE
thirty days → DATE
COMMISSION → ORG
CONSULTANT → ORG
COMMISSION → ORG
COMMISSION → ORG
COMMISSION → ORG
ten (10) days → DATE
120) days → DATE
COMMISSION → ORG
DAMAGES → ORG
TAXES → ORG
COMMISSION → ORG
COMMISSION → ORG
Independent CONSULTANT’s Insurance → ORG
A. Types of Insurance and Minimum Limits → ORG
1,000,000 → MONEY
Commercial General Liability Insurance → ORG
one
million dollars → MONEY
1,000,000 → MONEY
CSL

In [ ]:
# Save to a text file
with open("legal_summary_and_clauses.txt", "w") as f:
    f.write("🧠 SUMMARY:\n" + final_summary + "\n\n")
    f.write("📌 KEY CLAUSES:\n" + clause_data + "\n")
    f.write("🔍 NAMED ENTITIES:\n" + entities)

# Download the file
files.download("legal_summary_and_clauses.txt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>